# <span style="color:gold">Algoritmi di Crittografia (2023/24)</span> 
## Notebook 5

In [ ]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('/home/mauro/.jupyter/custom/custom.css').read()))

Latex definitions
$\DeclareMathOperator*{\prob}{prob}$
$\DeclareMathOperator*{\mod}{mod}$
$\DeclareMathOperator*{\ln}{ln}$
$\def\zn{\mathbf{Z}_n}$
$\def\zp{\mathbf{Z}_p}$
$\def\mcd{\mathrm{MCD}}$

In [1]:
# Import delle funzioni/moduli utilizzati nel notebook
from Crypto.PublicKey import RSA
from Crypto.Random.random import randint
from Crypto.Random import get_random_bytes
from Crypto.Util import number
from Crypto.Cipher import AES, PKCS1_OAEP
from math import sqrt

### <span style="color:gold">Il sistema crittografico a chiave pubblica RSA</span>
* Il sistema crittografico <span style="color:gold">RSA</span> prende il nome dalle iniziali dei loro inventori, Ronald L. Rivest, Adi Shamir e Leonard M. Adlemam, che nel 2002 hanno ricevuto il Premio Turing: <span style="color:gold">for their ingenious contribution to making public-key cryptography useful in practice</span>, di fatto proprio per l'invenzione dell'RSA.
* RSA basa la sua efficacia, in termini di sicurezza, sulla difficoltà computazionale del <span style="color:gold">problema della fattorizzazione di numeri interi</span>, ovvero sul fatto che ad oggi non esistono algoritmi efficienti per fattorizzare numeri interi di grandi dimensioni.
* In realtà, ad oggi, <span style="color:gold">non esiste argomentazione matematica</span> che dimostri che saper violare efficientemente RSA implichi saper anche efficientemente fattorizzare numeri interi
* Non è tuttavia nemmeno noto un procedimento alternativo alla fattorizzazione per violare il codice e si ritiene che non ci sia
* Un sistema crittografico <span style="color:gold">equivalente alla fattorizzazione</span> è stato invece proposto (solo poco dopo la pubblicazione dell'RSA) da un altro premio Turing, l'israeliano, Michael O. Rabin; lo vedremo dopo aver analizzato RSA
* Come ultima nota generale, ricordiamo che un <span style="color:gold">computer quantistico</span> sarebbe in grado di fattorizzare efficientemente i numeri interi e dunque anche violare un sistema crittografico basato sull'algoritmo RSA

### <span style="color:cyan">Il protocollo base (Textbook RSA)</span>
* Descriviamo ora l'<span style="color:gold">algoritmo base</span> e la "matematica" che ci sta dietro
* Vedremo esempi di generazione delle chiavi e di uso del protocollo di cifratura. 
* Analizzeremo poi alcune <span style="color:gold">potenziali debolezze</span> e vedremo (a grandi linee) come viene inserito in un <span style="color:gold">protocollo reale</span> in grado di fornire garanzie di sicurezza molto maggiori rispetto al protocollo base.

#### <span style="color:cyan">L'algoritmo in dettaglio</span>

* Come in tutti i protocolli asimmetrici, la generazione delle chiavi è eseguita dal <span style="color:gold">destinatario dei messaggi cifrati</span>, che supporremo sia Alice. 
* Il protocollo ha <span style="color:gold">un solo parametro di input</span>, che è la lunghezza in bit, indicata con $N$, dell'intero con cui si effettueranno le riduzioni modulari. 
* Vediamo i dettagli dell'algoritmo.

<br />

#### <span style="color:cyan">Generazione delle chiavi (Alice)</span>
1. genera <span style="color:gold">due numeri primi a caso</span>, $p$ e $q$, di lunghezza $N/2$ bit;
2. calcola il <span style="color:gold">prodotto $n=p\cdot q$</span>;
3. calcola la quantità <span style="color:gold">$\phi(n)=(p-1)\cdot(q-1)$</span>;
4. determina (in modo sostanzialmente arbitrario), un <span style="color:gold">intero $e$ relativamente primo con $\phi(n)$</span>, tale cioè che $\mathrm{MCD}(e,\phi(n))=1$;
5. calcola l'<span style="color:gold">inverso moltiplicativo $d$</span> di $e$ modulo $\phi(n)$, che esiste proprio perché $e$ è relativamente primo con $\phi(n)$;
6. diffonde la coppia <span style="color:gold">$(e,n)$ come chiave pubblica</span> e conserva <span style="color:gold">$d$ come chiave segreta</span>.

<br />

#### <span style="color:cyan">Cifratura di un messaggio $M$ (Bob)</span>
1. si procura la chiave pubblica $(e,n)$ di Alice;
2. calcola <span style="color:gold">$C=M^e\ \mathrm{mod}\ n$</span>;
3. invia $C$ ad Alice come messaggio cifrato</li>

<br />

#### <span style="color:cyan">Decifrazione del messaggio $C$ (Alice)</span>
1. calcola <span style="color:gold">$M=C^d\ \mathrm{mod}\ n$</span>

* Prima di dimostrare la correttezza del protocollo, vediamo qualche semplice esempio

### <span style="color:cyan">Qualche esempio illustrativo</span>

In [40]:
from ACLIB.utils import modular_inverse, modexp, getprime

In [28]:
def rsakeys(p,q):
    '''Returns pub and sec keys, e and d'''
    from ACLIB.utils import modular_inverse
    from random import randint
    phi = (p-1)*(q-1)
    e = 2*randint(1,phi>>1)+1
    while True:
        if e%p == 0 or e%q == 0:
            e = 2*randint(1,phi>>1)+1
            continue
        d = modular_inverse(e,phi) 
        if d is None or d == e:
            e = 2*randint(1,phi>>1)+1
            continue
        return e,d

#### <span style="color:cyan">Numeri molto piccoli</span>

In [12]:
p = 19
q = 31
n = p*q
phi = (p-1)*(q-1)
e,d = rsakeys(p,q)

In [13]:
e,d,n,phi

(23, 47, 589, 540)

In [14]:
(e*d)%phi

1

In [15]:
M = randint(1,n-1); M 

412

In [16]:
C = (M**e)%n; C

90

In [17]:
(C**d)%n

412

#### <span style="color:cyan">Numeri piccoli per scopi crittografici</span>

In [29]:
from Crypto.Util import number

In [43]:
N = 256    # Lunghezza in bit del modulo

In [44]:
p = number.getPrime(N//2); p

222830599487371856893424612355349850299

In [45]:
q = number.getPrime(N//2); q

187826365558767392152053768148528575479

In [46]:
n = p*q
phi = (p-1)*(q-1)
e,d = rsakeys(p,q)
print(e,d,n)

39006778740733519911284238769189477943810633268029490928628277709978425518129 15376304998659002759727650172186641772907028994395316337836673176771817886729 41853461636994392250829679061626014052158062520668353899216772841586372218221


In [47]:
M = randint(1,n-1); M 

9247481978863759017939702120615014557634641529603457080076812633448264859746

In [48]:
C = modexp(M,e,n); C

32273572003840310856543046487022381158721195186742039461809810717647849708442

In [49]:
D = modexp(C,d,n)
D == M

True

### <span style="color:magenta">Key management in Python con Crypto</span>

In [ ]:
from Crypto.PublicKey import RSA

#### <span style="font-style: italic; color:cyan">Generazione delle chiavi</span> 

In [ ]:
key = RSA.generate(2048)     # Generazione della chiave

In [ ]:
# Parametri fondamentali
p = key.p
q = key.q
n = key.n
e = key.e
d = key.d

In [ ]:
print(f"Public exponent e={e}")

In [ ]:
e==2**16+1  # 2**16+1 = 10000000000000001

La chiave pubblica, naturalmente, include solo i parametri pubblici

In [ ]:
public_key = key.publickey()

In [ ]:
public_key.n

In [ ]:
public_key.e

In [ ]:
public_key.d

#### <span style="color:cyan">Esportazione e importazione delle chiavi</span> 

* Esportazione della chiave pubblica

In [ ]:
# Per esportare la chiave pubblica: key.publickey().exportKey()
with open('mypubkey.pem','wb') as f:
    f.write(public_key.exportKey())

* Importazione della chiave pubblica

In [ ]:
imported_pub_key = RSA.importKey(open("mypubkey.pem").read())

In [ ]:
imported_pub_key == public_key

In [ ]:
public_key

#### <span style="color:cyan">Esportazione ed importazione della chiave privata</span>

In [ ]:
# Per esportare la chiave privata è sufficiente: key.exportKey()
# Questo perché la chiave è la chiave privata! La chiave pubblica 
# invece è un "sottoinsieme" che deve prima essere "estratto" 
# usando il metodo publickey()
# E' però altamente consigliabile proteggere le chiave segreta con
# una strong passphrase
privateKey = \
key.exportKey(passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')
with open('myprivkey.pem','wb') as f:
    f.write(privateKey)

In [ ]:
with open("myprivkey.pem", "wb") as f:
    f.write(privateKey)

In [ ]:
imported_priv_key = \
   RSA.importKey(open("myprivkey.pem").read(),passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')

In [ ]:
imported_priv_key==key

### <span style="color:gold">Correttezza del protocollo RSA</span>
* Ritorniamo ora alla questione lasciata in sospeso, e cioè la <span style="color:gold">correttezza del protocollo</span>
* Dobbiamo dimostrare che effettivamente Alice, con il calcolo di $C^d\ \mathrm{mod}\ n$, rimette in chiaro il messaggio cifrato da Bob, ovvero che vale <span style="color:gold">$C^d\ \mathrm{mod}\ n=M$</span>

* In uno dei passaggi algebrici, useremo la seguente uguaglianza: 
$$
ed = k\cdot \phi(n)+1
$$
per un qualche intero $k$. 
* Ricordiamo al riguardo che $d$ è l'inverso moltiplicativo di $e$ (modulo $\phi(n)$), cioè che vale <span style="color:gold">$e\cdot d\ \mathrm{mod}\ \phi(n)=1$</span>. 
* Questo vuol proprio dire che il prodotto <span style="color:gold">$ed-1$ è un multiplo di $\phi(n)$</span>.

* Abbiamo dunque

\begin{eqnarray*}
C^d\ \mathrm{mod}\ n&=&\left(M^e\ \mathrm{mod}\ n\right)^d\ \mathrm{mod}\ n\\
&=&M^{e\cdot d}\ \mathrm{mod}\ n\\
&=&M^{k\cdot \phi(n)+1}\ \mathrm{mod}\ n\\
&=&M\cdot M^{k\cdot \phi(n)}\ \mathrm{mod}\ n\\
&=&M\cdot M^{k\cdot(p-1)\cdot(q-1)}\ \mathrm{mod}\ n
\end{eqnarray*}

* A questo punto dimostriamo preliminarmente che <span style="color:gold">vale l'uguaglianza</span>:

$$
M\cdot M^{k\cdot(p-1)\cdot(q-1)}\ \mathrm{mod}\ p = M\ \mathrm{mod}\ p
$$

* L'uguaglianza è immediatamente verificata se $M\ \mathrm{mod}\ p= 0$, perché in tal caso <span style="color:gold">entrambi i membri sono ovviamente 0</span>

* In caso contrario, cioè <span style="color:gold">se $M\ \mathrm{mod}\ p\neq 0$</span>, abbiamo

\begin{eqnarray*}
\left(M\cdot M^{k\cdot(p-1)\cdot(q-1)}\right)\ \mathrm{mod}\ p&=&\left(M\cdot \left(M^{p-1}\right)^{k\cdot(q-1)}\right)\ \mathrm{mod}\ p\\
&=&\left(M\cdot \left(M^{p-1}\ \mathrm{mod}\ p\right)^{k\cdot(q-1)}\ \right) \mathrm{mod}\ p\\
&=&\left(M\cdot \left(1\right)^{k\cdot(q-1)}\right)\ \mathrm{mod}\ p\\
&=&M\ \mathrm{mod}\ p
\end{eqnarray*}
* Allo stesso modo <span style="color:gold">vale evidentemente</span> 
$$
M\cdot M^{k\cdot(p-1)\cdot(q-1)}\ \mathrm{mod}\ q = M\ \mathrm{mod}\ q
$$

* Dunque vale simultaneamente <span style="color:gold">$C^d\ \mathrm{mod}\ p= M\ \mathrm{mod}\ p$</span> e <span style="color:gold">$C^d\ \mathrm{mod}\ q= M\ \mathrm{mod}\ q$</span>.
* In altri termini le quantità $C^d$ e $M$ hanno <span style="color:gold">gli stessi resti</span> nella divisione per $p$ e per $q$.
* Ma allora, necessariamente poiché $p$ e $q$ sono relativamente primi e $n=pq$ (si ricordi il <span style="color:gold">Teorema cinese dei resti</span>), vale anche $C^d\ \mathrm{mod}\ n = M\ \mathrm{mod}\ n$

#### <span style="color:gold">Efficienza</span>

* La relativa "abbondanza" dei numeri primi e la <span style="color:gold">mancanza di ulteriori requisiti forti</span> (tipo che il numero generato sia un safe prime) fa sì che $p$ e $q$ possano  essere determinati velocemente.
* Il calcolo di $e$ e poi del suo inverso sono eseguiti una volta sola
* Cifratura e decifrazione richiedono il calcolo di esponenziali modulari. In se si tratta di operazioni <span style="color:gold">efficientemente implementate</span> ma che coinvolgono numeri non gestibili in aritmetica di macchina.
* Sulla scelta di $e$ ritorneremo più avanti in relazione alla sicurezza. Qui ci interessa capire come si può procedere per <span style="color:gold">trovare un intero coprimo con $\phi(n)$</span>. 
* Una semplice scelta consiste nel prendere <span style="color:gold">$e$ primo e maggiore di $\max\{p,q\}$</span>. 
* Infatti $\phi(n)=(p-1)\cdot(q-1)$ e dunque i suoi fattori primi sono  inferiori a $p$ e $q$. 
* Altrimenti si può procedere per <span style="color:gold">tentativi "casuali"</span>, grazie al fatto che il calcolo del MCD è molto veloce. 

* Un'ulteriore possibilità consiste nel fissare a priori un <span style="color:gold">esponente piccolo (es 3 o 5)</span> e continuare a generare i primi $p$ e $q$ fino a quando non si verifica la condizione <span style="color:gold">$\mathrm{MCD}(e,(p-1)\cdot(q-1))=1$</span>. 
* Questo approccio ha alcuni vantaggi in ordine alla velocizzazione del calcolo di $M^e\ \mathrm{mod}\ n$, cosa che si potrebbe rivelare utile in <span style="color:gold">applicazioni della firma digitale</span> (come vedremo). 
* Nelle implementazioni "reali" del protocollo si procede in questo modo ma il valore di $e$ che viene scelto è <span style="color:gold">più elevato</span>.

#### <span style="color:gold">Sicurezza</span>

* Dal punto di vista "strettamente matematico", la sicurezza di RSA risiede nella <span style="color:gold">difficoltà di fattorizzare numeri interi</span> di grandi dimensioni. 
* Se la fattorizzazione fosse facile (cioè se disponessimo di un algoritmo di fattorizzazione efficiente) allora RSA sarebbe violabile perché, recuperando $p$ e $q$, <span style="color:gold">Eva potrebbe calcolare $\phi(n)$</span> e quindi anche $d$. 
* Il <span style="color:gold">viceversa non è dimostrato</span> ma, in più di quarant'anni di sforzi, non è stato trovato un metodo efficiente per violare RSA, che quindi non passi attraverso la fattorizzazione del modulo $n$.
* Altro faccenda è però <span style="color:gold">come vada implementato RSA</span> in modo che non presti il fianco ad attacchi "vincenti".

### <span style="color:cyan">Alcuni possibili problemi (generali e/o specifici del protocollo base)</span>

#### <span style="color:cyan">Malleabilità</span>

* Ricordiamo che un algoritmo di cifratura $\mathbf{E}$ <span style="color:gold">è malleabile </span> se, dato un testo cifrato $C_1 = \mathbf{E}(M_1)$, è possibile creare un secondo testo cifrato $C_2$ tale che $C_2=\mathbf{E}(M_2)$ ed $M_2$ sia in <span style="color:gold">relazione "significativa"</span> con $M_1$ (per semplicità, nella notazione abbiamo omesso di indicare la chiave). 
* Il protocollo <span style="color:gold">RSA di base è malleabile</span>. 
* Infatti, il prodotto di due messaggi cifrati corrisponde al <span style="color:gold">prodotto dei due messaggi in chiaro</span>.
* Questo fatto presta il fianco al seguente attacco nel modello <span style="color:gold">Chosen ciphertext</span>.

* Sia $C_1 = M_1^e\ \mathrm{mod}\ n$ il testo cifrato. 
* Creiamo un <span style="color:gold">testo cifrato "intermedio"</span> $C_I=2^e\ \mathrm{mod}\ n$. 
* Questo naturalmente possiamo farlo perché la chiave $e$ è pubblica.
* Ora creiamo $C_2$ nel modo seguente: 
$$C_2 = C_1\cdot C_I
$$
* Questo è il nostro <span style="color:gold">chosen ciphertext</span>. 
* Se riusciamo, con una qualsiasi ragione, a <span style="color:gold">far decifrare $C_2$</span> abbiamo facilmente modo di ottenere $M_1$. Infatti:
\begin{eqnarray*}
C_2^d\ \mathrm{mod}\ n&=& (C_1\cdot C_I)^d\ \mathrm{mod}\ n\\
&=& \left(\left(M_1^e\ \mathrm{mod}\ n\right)\cdot \left(2^e\ \mathrm{mod}\ n\right)\right)^d\ \mathrm{mod}\ n\\
&=&\left(\left(M_1^e\right)^d\cdot \left(2^e\right)^d\right)\ \mathrm{mod}\ n\\
&=&(M_1\cdot 2)\ \mathrm{mod}\ n
\end{eqnarray*}

<br />
* Basta quindi un <span style="color:gold">semplice shift del messaggio decifrato</span> per ottenere quello originale

#### <span style="color:cyan">Caso di $e$ troppo piccolo</span>

* Un secondo problema occorre quando l'esponente <span style="color:gold">$e$ è piccolo e i messaggi sono a loro volta brevi</span>
* Ad esempio, se $e=3$ e il messaggio è breve, può acccadere che <span style="color:gold">$M^3<n$</span>. 
* In questo caso <span style="color:gold">la riduzione modulo $n$ è inutile</span> e dunque $C=M^3$. 
* In questo caso Eva può rimettere in chiaro il messaggio semplicemente <span style="color:gold">calcolando la radice cubica di $C$</span>.
* Per questa ragione, lo standard FIPS prevede che $e$ sia <span style="color:gold">non minore di $2^{16}+1=65537$</span>.

#### <span style="color:cyan">Possibilità di fattorizzare $n$</span>

* Esiste poi tutta una "gamma" di problemi legati alla <span style="color:gold">possibilità di fattorizzare $n$</span> se i numeri primi $p$ e $q$ hanno determinate caratteristiche "sbagliate"
* Presentiamo qui un caso che non richiede particolare sofisticazione matematica
* Il caso riguarda la situazione in cui <span style="color:gold">$p$ e $q$ sono "molto vicini"</span> (affermazione che ovviamente dovremo andare a quantificare)
* Ricordiamo solo <span style="color:gold">due semplici fatti</span>, sicuramente già noti.
    1. Se $n=p\cdot q$, e se $p\neq q$, allora necessariamente (assumendo senza perdita di generalità che $p>q$) vale
$$
q<\sqrt{n}<p
$$ 
    2. Il prodotto di due numeri dispari, come è il caso di $n$, può sempre essere <span style="color:gold">espresso come differenza di quadrati</span> di numeri interi. Posto infatti $n = p\cdot q$ (con $p$ e $q$ interi dispari) risulta
$$
n = \left(\frac{p+q}{2}\right)^2-\left(\frac{p-q}{2}\right)^2=\frac{1}{4}\left((p+q)^2-(p-q)^2\right)
$$
    *  Ad esempio: $11\cdot 7=77=9^2-2^2$

* Se dunque riuscissimo ad esprimere il modulo RSA $n$ come differenza di quadrati
$$
n = x^2-y^2=(x+y)(x-y)
$$
necessariamente (cioè poiché $p$ e $q$ sono primi) avremmo <span style="color:gold">$p=x+y$</span> e <span style="color:gold">$q=x-y$</span>
* Nel caso appena visto $77 = (9+2)(9-2) = 11\cdot 7$
* Nel caso almeno uno dei due fattori sia a sua volta un numero composto si possono avere anche <span style="color:gold">decomposizioni diverse</span> (naturalmente)
* Ad esempio, nel caso di $n=945$ abbiamo
$$
n = 27\cdot 35 = 31^2 - 4^2
$$
ma anche
$$
n = 105\cdot 9 = 57^2-48^2
$$

* Volendo portare un attacco a RSA tramite la fattorizzazione di $n$ dobbiamo chiederci con quanta facilità si possono trovare le due quantità $\frac{p+q}{2}$ e $\frac{p-q}{2}$, ovviamente <span style="color:gold">senza la conoscenza di $p$ e $q$</span>.
* L'idea parte dalla riscrittura di $n = x^2 - y^2$ nel modo seguente
$$
x^2 - n = y^2
$$
che suggerisce la <span style="color:gold">strategia brute force</span> illustrata di seguito
1. Si inizia con un certo valore $x=x_0$ e si calcola <span style="color:gold">$z_0=x_0^2-n$</span>. 
2. Se $z_0$ è un quadrato perfetto, stop, abbiamo finito, altrimenti procediamo incrementando il valore $x_0$
4. In generale, <span style="color:gold">al generico passo $i$</span>, posto $x_i=x_0+i$, se <span style="color:gold">$z_i = x_i^2-n$</span> è un quadrato perfetto, stop, altrimenti procediamo con un ulteriore incremento

* Dobbiamo ovviamente chiarire alcuni punti. (1) Come <span style="color:gold">scegliere il valore $x_0$</span> e (2) qual è <span style="color:gold">la complessità</span> di questo procedimento.

* Riguardo il punto 1, dall'uguaglianza
$$
n = \left(\frac{p+q}{2}\right)^2-\left(\frac{p-q}{2}\right)^2
$$
ricaviamo ovviamente
$$
\left(\frac{p+q}{2}\right)^2 - n = \left(\frac{p-q}{2}\right)^2
$$
* &Egrave; chiaro quindi che il valore iniziale <span style="color:gold">$x_0$ deve essere non maggiore di $\frac{p+q}{2}$</span> perché altrimenti avremmo $z_0=x_0^2-n>\left(\frac{p+q}{2}\right)^2 - n =  \left(\frac{p-q}{2}\right)^2$ e poiché il valore di $x_i$ (e dunque di $z_i$)  aumenta ad ogni passo, l'<span style="color:gold">uguaglianza non verrebbe mai soddisfatta</span>
* &Egrave; poi facile verificare che $p\neq q$ e $p\cdot q = n$ implica <span style="color:gold">$\frac{p+q}{2}>\sqrt{n}$</span> (basta elevare al quadrato ed eseguire pochi passaggi algebrici).
* Dunque il punto di partenza dell'algoritmo può essere proprio $x_0=\sqrt{n}$, o meglio $x_0=\left\lceil\sqrt{n}\right\rceil$.

* Possiamo quindi scrivere l'algoritmo completo

1. Dato $n$, prodotto di due numeri dispari distinti, poni $x = \left\lceil\sqrt{n}\right\rceil$
2. Calcola $z = x^2-n$
3. Se $z$ non è un quadrato perfetto, poni $x=x+1$ e ritorna al passo 2
4. Calcola e restituisci $x+\sqrt{z}$ e $x-\sqrt{z}$

In [50]:
from ACLIB.utils import intsqrt
def RSAfactorize(n,printiter=False):
    '''n deve essere il prodotto di due interi dispari. Usa il metodo di Fermat'''
    x = intsqrt(n)+1   # Guess iniziale
    z = x*x-n         
    s = intsqrt(z)
    i = 0
    while s*s != z:   # Se z è un quadrato perfetto, stop
        i += 1
        z += (x<<1)+1 # Altrimenti calcoliamo z = (x+1)(x+1)-n (= z attuale+2x+1)
        x += 1
        s = intsqrt(z)
    if printiter:
        print(f"Numero di iterazioni: {i}")
    return x+s,x-s

In [51]:
from Crypto.Util import number

In [52]:
n = number.getPrime(32)*number.getPrime(32)
print(f"Numero da fattorizzare: {n}")
p,q=RSAfactorize(n)
print(p,q,p*q==n)

Numero da fattorizzare: 8427322075335414049
2973192269 2834435621 True


<span style="color:cyan">Efficienza del metodo</span>
* Rimane ancora da discutere la questione di quando sia fattibile applicare l'algoritmo, ovvero sotto quali condizioni esso riesce a fattorizzare l'input in <span style="color:gold">"tempi ragionevoli"</span>. 
* Abbiamo già anticipato che questo motodo ha chance di successo (su numeri di grandi dimensioni) se <span style="color:gold">$p$ e $q$ sono vicini</span>. &Egrave; il momento di quantificare l'affermazione.

* Supponiamo che valga la seguente diseguaglianza:
$$
\frac{p-q}{2}<c\cdot\sqrt{2\cdot q}
$$
con $c$ costante. 
* Dall'uguaglianza da cui siamo partiti per lo sviluppo dell'algoritmo, e cioè
$$
\left(\frac{p+q}{2}\right)^2 -n = \left(\frac{p-q}{2}\right)^2
$$
possiamo ricavare
$$
\left(\frac{p+q}{2}+\sqrt{n}\right)\cdot \left(\frac{p+q}{2}-\sqrt{n}\right) = \left(\frac{p-q}{2}\right)^2
$$
* Poiché $q<\sqrt{n}<p$, possiamo <span style="color:gold">minorare con $2q$</span> il primo fattore a sinistra (sostituendo cioè sia $p$ che $\sqrt{n}$ con $q$
* Per ipotesi, poi, possiamo <span style="color:gold">maggiorare il membro destro con $2qc^2$</span> e dunque, in definitiva, abbiamo
$$
2\cdot q\cdot\left(\frac{p+q}{2}-\sqrt{n}\right)<2\cdot q\cdot c^2
$$

ovvero

$$
\frac{p+q}{2}-\sqrt{n}<\frac{2\cdot q\cdot c^2}{2\cdot q}=c^2
$$

* Questo è il risultato desiderato perché la quantita a sinistra è proprio il <span style="color:gold">numero passi (iterazioni)</span> chel'algoritmo impiega per arrivare alla soluzione e la diseguaglianza dice che questo è limitato da $c^2$. 
* In concreto, però, che cosa significa la assunzione che abbiamo fatto per ottenere questo risultato? Ovvero che:

$$
\frac{p-q}{2}<c\cdot\sqrt{2\cdot q}
$$
* Poiché $q$ è $O(\sqrt{n})$, la nostra assunzione implica che la <span style="color:gold">differenza fra $p$ e $q$ risulta $O(\sqrt[4]{n})$</span>. 
* In termini di cifre, questo a sua volta significa che <span style="color:gold">$p$ e $q$ coincidono nella metà più significativa</span>.

Vediamo qualche esempio

In [53]:
from ACLIB.utils import intsqrt
from math import sqrt
p = number.getPrime(1024)
q = (p - 10*intsqrt(p))|1  # q è tentativamente p-10 sqrt(p)
while not number.isPrime(q):
    q -= 2
print(f"Fattore p (maggiore): {p}")
print(f"Fattore q (minore):   {q}")
print(f"Differenza:           {p-q}")
# Numero di passi previsto
f = 1/(2*intsqrt(2*q))
c = int((p-q)*f)
print(f"Numero approssimativo di passi:", c*c)

Fattore p (maggiore): 114099927554896871879548980720201261707937674703108534397672480387799844431550329261041357341718423200057232012758139397229646029283841266206212751151228190979548286459675245097928994346958536102679365149730382339868683773750640693158532571790382638672817565993494990525163878080569721802636269504535565510281
Fattore q (minore):   114099927554896871879548980720201261707937674703108534397672480387799844431550329261041357341718423200057232012758139397229646029283841266206212751151228084161980651310412837069516708203087991126880428306232773487901855438989104541037479123069616006001313775718420718006432142028615483574457593907789707329603
Differenza:           106817567635149262408028412286143870544975798936843497608851966828334761536152121053448720766632671503790275074272518731736051954238228178675596745858180678
Numero approssimativo di passi: 9


In [54]:
n = p*q
r,s = RSAfactorize(n,printiter=True)
print(r,s,r*s == n)

Numero di iterazioni: 12
114099927554896871879548980720201261707937674703108534397672480387799844431550329261041357341718423200057232012758139397229646029283841266206212751151228190979548286459675245097928994346958536102679365149730382339868683773750640693158532571790382638672817565993494990525163878080569721802636269504535565510281 114099927554896871879548980720201261707937674703108534397672480387799844431550329261041357341718423200057232012758139397229646029283841266206212751151228084161980651310412837069516708203087991126880428306232773487901855438989104541037479123069616006001313775718420718006432142028615483574457593907789707329603 True


In [6]:
p==r

True

#### <span style="color:cyan">Vulnerabilità dipendenti dai generatori casuali</span>

* In uno studio condotto nel 2012, sono stati analizzati <span style="color:gold">4.7 milioni di moduli RSA</span> di 1024 bit. 
* &Egrave; risultato che <span style="color:gold">i duplicati non sono infrequenti</span>. 
* In 12720 casi, i moduli avevano un <span style="color:gold">fattore primo in comune</span>. 
* Per quanto riguarda la duplicazione del modulo, vedremo come sia possibile utilizzare questa circostanza per ricostruire le corrispondenti chiavi private. 
* Riguardo l'altra situazione, il fatto cioè che due moduli abbiano un fattore primo in comune, è immediato vedere come questo renda <span style="color:gold">vulnerabile il protocollo</span>. 
* Infatti, il fattore in comune può essere scoperto calcolando il <span style="color:gold">MCD dei due moduli</span>. 
* Da questo e dal modulo si può poi banalmente risalire alla determinazione dell'altro fattore.

$$
n_1 = p\cdot q_1,\quad\mathrm{e}\quad n_2 = p\cdot q_2\qquad\Rightarrow\qquad p = \mathrm{MCD}(n_1,n_2)
$$

* La causa di queste situazioni estremamente pericolose è da ascrivere primariamente a <span style="color:gold">debolezze dei generatori casuali</span> coinvolti nella scelta dei numeri primi.
* Riguardo la condivisione del modulo nella sua interezza esiste in letteratura tutta una serie di possibili attacchi che sfruttano altre <span style="color:gold">debolezze</span>, vuoi di <span style="color:gold">natura tecnica</span> (es. esponente privato relativamente piccolo), vuoi legate ad un <span style="color:gold">possibile scenario applicativo</span> "azzardato".
* Nel seguito riportiamo (per studio opzionale) un caso di quest'ultimo tipo

#### <span style="color:magenta">Un attacco al modulo comune</span>

* Consideriamo uno scenario applicativo in cui è presente un amministratore o, più in generale, una <span style="color:gold">trusted central authority</span>, e un insieme di <span style="color:gold">$k$ utenti</span>. 
* Supponiamo che tale authorithy, riconosciuta e affidabile, distribuisca agli utenti altrettante <span style="color:gold">coppie di chiavi</span>, $(e_i,d_i), i=1,\ldots,k$. 
* Le chiavi, condividono lo stesso modulo $n$, anch'esso distribuito agli utenti ma ovviamente <span style="color:gold">senza i fattori $p$ e $q$</span>.
* Questo può sembrare conveniente perché la condivisione del modulo può comportare <span style="color:gold">vantaggi di natura computazionale</span> e, a prima vista, nessun problema di sicurezza. 
* Tuttavia questo è un errore. Infatti, dalla conoscenza di <span style="color:gold">una coppia $e_j$ e $d_j$</span> si può <span style="color:gold">risalire alla fattorizzazione di $n$</span> e quindi alla determinazione di tutti gli esponenti privati.

* Con $n=p\cdot q$, consideriamo il gruppo $\mathbf{Z}_n^*$. 
* Gli elementi, come sappiano, sono gli interi minori di $n$ che sono primi con $n$ (perché solo questi hanno inverso). 
* Sappiamo anche che il numero di elementi è $\phi(n)=(p-1)\cdot(q-1)$. 
* Come esempio, proviamo a scrivere la <span style="color:gold">tabella del prodotto</span> per gli elementi del gruppo nel caso $n=p\cdot q=3\cdot 5=15$.

<table style="margin: 0 auto; border: 1px solid black;border-collapse: collapse;">
  <tr>
    <th></th>
    <th style="color:gold">1</th>
    <th style="color:gold">2</th>
    <th style="color:gold">4</th>
    <th style="color:gold">7</th> 	
    <th style="color:gold">8</th>
    <th style="color:gold">11</th>
    <th style="color:gold">13</th> 	
    <th style="color:gold">14</th>      
  </tr>
  <tr>
    <th style="color:gold">1</th>
    <td style="font-weight: bold; color:cyan">1</td>
    <td>2</td>
    <td>4</td>
    <td>7</td> 	
    <td>8</td>
    <td>11</td>
    <td>13</td> 	
    <td>14</td>
  </tr>  
  <tr>
    <th style="color:gold">2</th>
    <td>2</td>
    <td>4</td>
    <td>8</td>
    <td>14</td> 	
    <td>1</td>
    <td>7</td>
    <td>11</td> 	
    <td>13</td>    
  </tr>
  <tr>
    <th style="color:gold">4</th>
    <td>4</td>
    <td>8</td>
    <td style="font-weight: bold; color:cyan">1</td>
    <td>13</td> 	
    <td>2</td>
    <td>14</td>
    <td>7</td> 	
    <td>11</td>    
  </tr>
  <tr>
    <th style="color:gold">7</th>
    <td>7</td>
    <td>14</td>
    <td>13</td>
    <td>4</td> 	
    <td>11</td>
    <td>2</td>
    <td>1</td> 	
    <td>8</td>    
  </tr>
  <tr>
    <th style="color:gold">8</th>
    <td>8</td>
    <td>1</td>
    <td>2</td>
    <td>11</td> 	
    <td>4</td>
    <td>13</td>
    <td>14</td> 	
    <td>7</td>    
  </tr>
  <tr>
    <th style="color:gold">11</th>
    <td>11</td>
    <td>7</td>
    <td>14</td>
    <td>2</td> 	
    <td>13</td>
    <td style="font-weight: bold; color:cyan">1</td>
    <td>8</td> 	
    <td>4</td>    
  </tr>
  <tr>
    <th style="color:gold">13</th>
    <td>13</td>
    <td>11</td>
    <td>7</td>
    <td>1</td> 	
    <td>14</td>
    <td>8</td>
    <td>4</td> 	
    <td>2</td>    
  </tr>
  <tr>
    <th style="color:gold">14</th>
    <td>14</td>
    <td>13</td>
    <td>11</td>
    <td>8</td> 	
    <td>7</td>
    <td>4</td>
    <td>2</td> 	
    <td style="font-weight: bold; color:cyan">1</td>    
  </tr>
</table>

* Possiamo notare che nella diagonale della tabella ci sono <span style="color:gold">quattro 1</span>. 
* Questo vuol dire che ci sono quattro elementi del gruppo che <span style="color:gold">soddisfano l'equazione $x^2\ \mathrm{mod}\ 15=1$</span>, ovvero che sono <span style="color:gold">radici quadrate dell'unità</span>.
* Questa è una <span style="color:gold">proprietà generale</span> dei gruppi $\mathbf{Z}_n^*$ in cui <span style="color:gold">$n$ è il prodotto di due primi</span>. 
* In questo caso, <span style="color:gold">conoscendo i due primi $p$ e $q$</span>, è semplice trovare le quattro radici dell'unità, usando la seguente formula, che altro non è il <span style="color:gold">Teorema cinese dei resti</span> specializzato al caso di due primi:
$$
x = \left(\left(q\left(q^{-1}\mod p\right)\right)x_p+\left(p\left(p^{-1}\mod q\right)\right)x_q\right)\mod n
$$
dove $x_p,x_q\in\{(1,1), (1,q-1), (p-1,1), (p-1,q-1)\}$ sono <span style="color:gold">tutte le possibili coppie</span> in cui gli elementi sono, rispettivamante, radici quadrate dell'unità modulo $p$ e modulo $q$


* Verifichiamo la formula proprio con $p=3$ e $q=5$
* In questo caso abbiamo
$$
q\left(q^{-1}\mod p\right)=5\left(5^{-1}\mod 3\right)=5\cdot 2 = 10
$$
e
$$
p\left(p^{-1}\mod q\right)=3\left(3^{-1}\mod 5\right)=3\cdot 2 = 6
$$
da cui, usando <span style="color:gold">tutte le possibilità per $x_p$ e $x_q$</span> otteniamoeffettivamente le quattro radici quadrati dell'unità in $\mathbf{Z}_{15}^*$:
    1. $x_1 = (10\cdot 1+6\cdot 1)\mod 15 = 16\mod 15 = 1$
    2. $x_2 = (10\cdot 1+6\cdot 4)\mod 15 = 34\mod 15 = 4$
    3. $x_3 = (10\cdot 2+6\cdot 1)\mod 15 = 26\mod 15 = 11$
    4. $x_4 = (10\cdot 2+6\cdot 4)\mod 15 = 44\mod 15 = 14$

* Chiamiamo <span style="color:gold">banali</span> le due radici $x_1$ e $x_4$
* Le due radici non banali, $x_2$ e $x_3$, <span style="color:gold">corrispondono alle coppie $(1,q-1)$ e $(p-1,1)$</span>, ed è immediato verificare che:
    1. per $x_2$ vale $(x_2-1)\mod p=0$ e $(x_2-1)\mod q\neq 0$
    2. per $x_3$ vale l'esatto contrario, cioè $(x_3-1)\mod p\neq 0$ e $(x_3-1)\mod q=0$
* Questo vuol dire che <span style="color:gold">$x_2-1$ è multiplo di $p$ ma non di $q$ e dunque neppure di $n$</span>.
* Ma allora <span style="color:gold">$\mcd(x_2-1,n)=p$</span>
* Simmetricamente, <span style="color:gold">$\mcd(x_3-1,n)=q$</span>

* Se dunque conoscessimo un modo <span style="color:gold">efficiente per calcolare una radice non banale dell'unità</span>, potremmo calcolare un fattore di $n$ e, quindi, chiaramente, fattorizzare completamente il modulo
* Le radici banali invece non aiutano (e dovrebbe essere immediato il perché).
* In effetti, disponiamo di un <span style="color:gold">algoritmo randomizzato</span> che, usando $e$ e $d$, riesce a trovare una radice dell'unità di $\mathbf{Z}_n^*$. 
* L'algoritmo <span style="color:gold">non ha controllo</span> su quale sia la radice viene trovata, che potrebbe quindi anche essere una delle due banali 
* Tuttavia, poiché le radici non banali sono la metà, <span style="color:gold">ripetendo l'algoritmo</span> è ragionevole ritenere che con pochi tentativi si riesca a individuare una delle radici "buone".

#### <span style="color:magenta">Fattorizzazione di $n$ tramite $e$ e $d$</span>

* Nell'algoritmo facciamo uso della seguente proprietà, che abbiamo già usato nella <span style="color:gold">dimostrazione di correttezza dell'RSA</span>, e che qui riscriviamo per comodità in modo leggermente diverso

<br />
<div style="width:700px; margin:0 auto;">
Poiché $e\cdot d = k\cdot\phi(n)+1\ \mathrm{mod}\ n$, se <span style="color:gold">$z$ è un numero relativamente primo con $n$</span> risulta 
    $z^{e\cdot d-1}\ \mathrm{mod}\ n=z^{k\cdot(p-1)\cdot(q-1)}\ \mathrm{mod}\ n=1$
</div>

<br />

<ol>
    <li>Fattorizziamo <span style="color:gold">parzialmente</span> la quantità $e\cdot d-1 = k\cdot\phi(n)=k\cdot(p-1)\cdot(q-1)$ nel modo seguente:
        
$$
        e\cdot d-1 = 2^t\cdot r
$$
        
in cui <span style="color:gold">$t\geq 2$</span> perché $p-1$ e $q-1$ sono numeri pari e <span style="color:gold">$r$ è un numero dispari</span>. La fattorizziamo può essere ottenuta facilmente dividendo per due finché non c'è resto ($t$ è proprio il <span style="color:gold">numero di volte in cui abbiamo eseguito la divisione</span> per 2).
    </li>
    <li>
        Scegliamo un numero <span style="color:gold">$z$ a caso nell'insieme $\{1,2,\ldots,n-1\}$</span> e calcoliamo $\mcd(z,n)$. Se siamo così fortunati che tale valore è maggiore di 1, abbiamo evidentemente già trovato un divisore di $n$ (o $p$ o $q$) e dunque abbiamo finito. Altrimenti $z$ è primo con $n$ e per esso dunque vale $z^{e\cdot d-1}\ \mathrm{mod}\ n=1$. 
            </li>
    <li>
        Calcoliamo ora <span style="color:gold">$x=z^r\ \mathrm{mod}\ n$</span>. Questa quantità non può essere 1 perché $r$ è dispari. Tuttavia, poiché (come abbiamo appena visto) $x^{2^t}=(z^r)^{2^t}=z^{r2^t}=z^{e\cdot d-1}=1\mod n$, abbiamo la certezza che, partendo da $x$ ed <span style="color:gold">elevando ripetutamente al quadrato</span>, prima o poi otteniamo il valore 1
                   </li>
    <li>Posto $x_0=x$, calcoliamo cioè la sequenza di valori: <span style="color:gold">$x_{i+1}=x_i^2\mod n$, $i=0,1,...$</span> e ci fermiamo non appena troviamo $x_{i+1}=1$. In questo caso $\bar{x}=x_{i}$ è una radice quadrata dell'unità in $\zn^*$.
    </li>
    <li>
        Effettuiamo la verifica $\mcd(\bar{x}-1,n)$. Se questa quantità è maggiore di 1 e diversa da $n$, allora <span style="color:gold">$\bar{x}$ è una delle due radici non banali e abbiamo trovato uno dei due fattori di $n$</span>. Altrimenti $\bar{x}$ è una delle due radici banali e ripetiamo il procedimento scegliendo un altro valore di $z$.
</ol>

#### <span style="color:magenta">Implementazione in Python</span>

In [ ]:
from Crypto.Random.random import randint
def RSAattack(e,d,n):
    '''Restituisce i due fattori primi, p e q, tali che n=pq.
       Mostra come la condivisione del modulo fra piu coppie
       di esponenti (e_i,d_i) consenta, proprio attraverso
       la fattorizzazione di n, di ricostruire tutte le
       chiavi private che condividono il modulo.
    '''
    r = e*d-1
    t = 0
    while r&1==0:
        r = r>>1
        t+=1
    while True:
        z = randint(1,n-1)
        f = Euclid(z,n)
        if f>1 and f!=n:
            return f,n//f 
        x = modexp(z,r,n)
        x2 = modexp(x,2,n)
        while x2 != 1:
            x = x2
            x2 = modexp(x,2,n)
        f = Euclid(x-1,n)
        if f>1 and f!=n:
            return f,n//f

* Un esempio con chiavi reali

In [ ]:
from ACLIB.utils import Euclid,modexp

In [ ]:
key = RSA.generate(2048)
p = key.p
q = key.q
n = key.n
e = key.e
d = key.d

In [ ]:
p1,q1 = RSAattack(e,d,n)

In [ ]:
(p1==p and q1==q) or (p1==q and q1==p)

### <span style="color:cyan">Aspetti implementativi di RSA</span>

* Alla luce di quanto abbiamo visto, possiamo affermare che la sicurezza delle cifrature con l'algoritmo RSA richiede che siano considerati molti aspetti apparentemente marginali per <span style="color:gold">non incorrere in vulnerabilità</span> dall'effetto più o meno disastroso.
* In particolare, abbiamo visto quanto segue (che però non è tutto...)

1. I generatori casuali utilizzati per generare i fattori primi del modulo devono certamente essere ottimi dal punto di vista teorico. Questo però non basta; la generazione deve essere effettuata dopo che sia stata <span style="color:gold">raccolta sufficiente entropia</span> da garantire che le eventuali collisioni non abbiano probabilità maggiore di quanto previsto teoricamente.
2. I fattori primi $p$ e $q$ <span style="color:gold">non devono essere troppo vicini</span>, per evitare l'attacco basato sull'algoritmo di fattorizzazione di Fermat.
3. Moduli condivisi devono essere <span style="color:gold">sistematicamente evitati</span>.
4. Gli esponenti <span style="color:gold">$e$ e $d$ non devono essere piccoli</span> da rendere inefficace la riduzione modulo $n$.
5. La <span style="color:gold">malleabilità</span> del protocollo deve essere eliminata con opportuni accorgimenti.

* Ci occuperemo ora proprio di quest'ultimo aspetto, che non abbiamo ancora preso in considerazione. 
* Prima però, discutiamo un paio di questioni legati all'<span style="color:gold">efficienza computazionale</span>, perché anch'essa fa ovviamente parte degli obiettivi di una implementazione reale.

#### <span style="color:cyan">Qualche "trucco" per migliorare l'efficienza di RSA</span>

* Il primo aspetto riguarda la scelta dell'<span style="color:gold">esponente pubblico $e$</span>. 
* La raccomandazione FIPS, come già sottolineato, è che esso sia non minore di 65537. 
* In molte implementazioni concrete la scelta cade <span style="color:gold">proprio su questo numero</span> perché si tratta di un intero primo la cui rappresentazione binaria comprende quasi esclusivamente cifre 0. 
* Infatti $65537 = 2^{16}+1$ e la rappresentazione binaria è $10000000000000001$. 
* Proprio la presenza di molte cifre 0 rende <span style="color:gold">più efficiente il calcolo delle esponenziali modulari</span>.

* Il secondo aspetto implementativo riguarda la <span style="color:gold">fase di decifrazione</span>. 
* Se $C$ è il messaggio cifrato, l'algoritmo (come sappiamo) prevede di calcolare <span style="color:gold">$M = C^d\ \mathrm{mod}\ n$</span>. 
* La decifrazione può essere svolta in modo più efficiente usando il <span style="color:gold">teorema cinese dei resti</span> e riducendo poi la <span style="color:gold">dimensione degli esponenti</span>
    1. Per il teorema cinese dei resti sappiamo che, disponendo delle due quantità
    $$
    M_p = C^d\ \mathrm{mod}\ p\qquad\mathrm{e}\qquad M_q = C^d\ \mathrm{mod}\ q,
    $$
    possiamo risalire al valore $M$ nel modo seguente:
    $$
    M = \left(\left(q\cdot(q^{-1}\ \mathrm{mod}\ p)\right)M_p + \left(p\cdot(p^{-1}\ \mathrm{mod}\ q)\right)M_q\right)\ \mathrm{mod}\ n
    $$
    in cui i coefficienti $q\cdot(q^{-1}\ \mathrm{mod}\ p)$ e $p \cdot(p^{-1}\ \mathrm{mod}\ q)$ possono essere <span style="color:gold">precalcolati</span>
    2. Tuttavia, anziché calcolare $M_p$ e $M_q$ nel modo ovvio, determiniamo preliminarmente le quantità <span style="color:gold">$s=d\ \mathrm{mod}\ (p-1)$</span> e <span style="color:gold">$t=d\ \mathrm{mod}\ (q-1)$</span>, anch'esse indipendenti da qualsiasi messaggio
        * Si noti che, per definizione, $d=s+a\cdot(p-1)$  (per un qualche intero $a$) e dunque <span style="color:gold">$C^d\mod p=C^{s+a\cdot(p-1)}\mod p=C^{s}\mod p$</span>.
        * Analogamente, <span style="color:gold">$y^d\ \mathrm{mod}\ q=y^{t}\mod p$</span>
        
    e quindi calcoliamo $M_p$ e $M_q$ nel modo seguente
    $$
    M_p = C^s\ \mathrm{mod}\ p\qquad\mathrm{e}\qquad M_q = C^t\ \mathrm{mod}\ q
    $$

* In questo modo, a parte le quantità che vengono calcolate preliminarmente (una sola volta), il calcolo richiede due esponenziazioni anziché una sola, ma su <span style="color:gold">numeri di lunghezza circa dimezzata</span>.
* Oltre a ciò vengono calcolati due prodotti e una somma, operazioni di costo asintotico inferiore

In [63]:
from ACLIB.utils import modprod, modular_inverse, modexp

In [58]:
key = RSA.generate(2048)
s = key.d%(key.p-1)
t = key.d%(key.q-1)
cp = key.q*modular_inverse(key.q,key.p)
cq = key.p*modular_inverse(key.p,key.q)

In [59]:
def RSAdecrypt(y,n,p,q,s,t,cp,cq):
    xp = modexp(y,s,p)
    xq = modexp(y,t,q)
    x = (modprod(cp,xp,n)+modprod(cq,xq,n))%n
    return x

In [60]:
M=2
y = modexp(M,key.e,key.n); y

9614904525763361037152644073964718173561964862027784609716977252374419131524183909709083944932481842441648529499093437778663325543434471600661044867718867742360303205233705457691383980022163411692005277396757744490874417078330346214300506038217929141579324613853985147956389217891670360463681401332184093077952941498924454789907318667443842220879633831228782790964943158184404487845331162409812620882546498703311791972953414670998992811352048668973408248408995588375653770656918055753004248864976543981031345886543639052194465344101015513212056021945023304152448117787921005724441372006494856966957203716872850190974

In [61]:
modexp(y,key.d,key.n)

2

In [64]:
RSAdecrypt(y,key.n,key.p,key.q,s,t,cp,cq)

2

In [65]:
from time import time

In [66]:
n = 10
startt = time()
for _ in range(n):
    RSAdecrypt(y,key.n,key.p,key.q,s,t,cp,cq)
endt = time()
print(f"Elapsed time: {endt-startt:.4f}")

Elapsed time: 7.2141


In [67]:
startt = time()
for _ in range(n):
    modexp(y,key.d,key.n)
endt = time()
print(f"Elapsed time: {endt-startt:.4f}")

Elapsed time: 28.6519


### <span style="color:cyan">Optimal Asymmetric Encryption Padding (OAEP)</span>

* Il modo in cui la cifratura RSA viene effettuata in concreto (non solo nell'RSA, ma praticamente in tutti gli algoritmi di cifratura, simmetrici o asimmetrici) include informazione addizionale chiamata <span style="color:gold">padding</span>. 
* Per RSA il padding è costituito da <span style="color:gold">sequenze (pseudo)casuali</span> e dunque, anche per questo scopo, è necessario disporre di generatori affidabili.
* Oltre al generatore, il funzionamento dello schema di padding OAEP necessita la disponibilità di <span style="color:gold">due funzioni hash crittografiche</span>, che indicheremo genericamente con $G$ e $H$.
* Se $N$ indica la lunghezza in bit del modulo, la lunghezza massima "utile" dei messaggi è $t=N-h-k-2$, dove $h$ e $k$ sono <span style="color:gold">parametri del protocollo</span>. 
* Per gli scopi che vedono l'utilizzo di RSA, tale "spazio" è in generale <span style="color:gold">più che sufficiente</span>, dato che la lunghezza tipica moduli, espressa in byte, è non inferiore a 256 
* Il seguente diagramma illustra il funzionamento di OAEP. 
* Il messaggio finale, visto come sequenza di bit (o di byte), viene poi <span style="color:gold">convertito nel numero $x$</span> che costituisce l'input per l'algoritmo RSA.

<img src="OAEP.jpg">

* La decifrazione segue il <span style="color:gold">percorso inverso</span>. 
* Dapprima (con l'algoritmo RSA) $x$ viene ricalcolato e poi <span style="color:gold">trasformato nella sequenza di byte $M$</span>. 
* La funzione $H$ può quindi essere applicata allo <span style="color:gold">stesso input $m_1$</span> della fase di cifratura. 
* La seconda operazione $\oplus$ in fase di cifratura aveva fornito $r_1=H(m_1)\oplus r$; <span style="color:gold">grazie alle proprietà di $\oplus$</span> possiamo ora ottenere <span style="color:gold">$r = r_1\oplus H(m_1)$</span>. 
* Allo stesso modo, da $m_1 = m\oplus G(r)$ possiamo ora ottenere <span style="color:gold">$m = m_1\oplus G(r)$</span>. 
* Se gli ultimi $h$ bit di $m$ non sono 0, il messaggio viene <span style="color:gold">rigettato</span>.

#### <span style="color:cyan">Un esempio "completo"</span>
* In questo esempio, il <span style="color:gold">receiver è Bob</span> e dunque è lui che genera le chiavi e le esporta

In [ ]:
################## Generazione delle chiavi ####################
from Crypto.PublicKey import RSA
key = RSA.generate(2048)
# Esportazione della chiave pubblica su file
with open('BobKey.pem','wb') as f:
    f.write(key.publickey().exportKey())
# Esportazione della chiave privata su file con password di protezione
privateKey = \
key.exportKey(passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')
with open('BobSecretKey.pem','wb') as f:
    f.write(privateKey)

In [ ]:
################## Cifratura del messaggio ######################
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
message = 'Questo è un messaggio'.encode('utf-8')
# Importazione della chiave del ricevente
BobKey = RSA.importKey(open("BobKey.pem").read())
# Creiamo un oggetto per la preparazione e la cifratura secondo lo schema OAEP-RSA
rsa = PKCS1_OAEP.new(BobKey)
# Generiamo ora una chiave simmetrica
symmetricKey = get_random_bytes(16)
# Cifriamo la chiave simmetrica
rsaEncryptedSymmKey = rsa.encrypt(symmetricKey)
# Cifriamo il messaggio usando AES con la chiave simmetrica
IV = get_random_bytes(16)
aes = AES.new(symmetricKey, AES.MODE_CFB, IV)
encMessage = IV+aes.encrypt(message)
# Inviamo la coppia formata dalla chiave simmetrica cifrata con RSA
# e il messaggio cifrato con la medesima chiave simmetrica
toBob = (rsaEncryptedSymmKey,encMessage)

In [ ]:
################## Decifrazione del messaggio ######################
# from Crypto.PublicKey import RSA
rsaEncryptedSymmKey,encMessage = toBob
# Recuperiamo la chiave RSA privata
with open('BobSecretKey.pem','r') as f:
    key = f.read()
privateKey = RSA.importKey(key,passphrase='_AIV3ryII5tr0ngIIIPa55w0rd_')
# La prima cosa da fare è decifrare il messaggio cifrato con RSA per recuperare la
# chiave simmetrica
rsa = PKCS1_OAEP.new(privateKey)
symmetricKey = rsa.decrypt(rsaEncryptedSymmKey)
# Recuperiamo l'IV e decifriamo ora il messaggio usando AES
IV = encMessage[:16]
aes = AES.new(symmetricKey, AES.MODE_CFB, IV)
decryptedMessage = aes.decrypt(encMessage)[16:]

In [ ]:
decryptedMessage.decode('utf-8')